Lectura de Dataframes

In [16]:
import pandas as pd
import numpy as np
import glob # Para encontrar archivos

In [15]:
df_historicos_2020 = pd.read_csv('data/raw/DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.csv')
df_contaminante_2020 = pd.read_csv('data/raw/DATOS HISTÓRICOS 2020_Contaminante.csv')
df_contaminante_2021 = pd.read_csv('data/raw/DATOS HISTÓRICOS 2021_Contaminante.csv')
df_historicos_2021 = pd.read_csv('data/raw/DATOS HISTÓRICOS 2021_TODAS ESTACIONES.csv')
df_historicos_2022 = pd.read_csv('data/raw/DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.csv')
df_historicos_2023 = pd.read_csv('data/raw/DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES.csv')
df_historicos_2024 = pd.read_csv('data/raw/DATOS HISTÓRICOS 2024_TODAS ESTACIONES.csv')
df_historicos_2025 = pd.read_csv('data/raw/DATOS HISTÓRICOS 2025_TODAS ESTACIONES.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.csv'

In [ ]:
def aplicar_filtros_de_calidad(df):
    """
    Aplica los filtros de calidad (Banderas y Rangos Físicos) a un DataFrame.

    Parámetros:
    - df (pd.DataFrame): El DataFrame consolidado de la función anterior.

    Retorna:
    - pd.DataFrame: El DataFrame con columnas "_limpio" para las variables de interés.
    """
    print("Iniciando Fase de Limpieza (Banderas y Rangos)...")
    
    # --- 1. Definición de Reglas ---

    # [cite_start]Banderas que marcan un dato como "Inválida" [cite: 5]
    banderas_invalidas = [
        'p', 'c', 'd', 'b', 'm', 'l', 'z', 'o', 
        's', 'r', 'e', 'a', 'f', 'h', 'n', 'x'
    ]

    # Variables a limpiar y sus columnas de flags (AJUSTA ESTOS NOMBRES)
    # Asumo que si la variable es 'PM2.5', su flag es 'Flag_PM2.5'
    mapeo_variables_flags = {
        'PM2.5': 'Flag_PM2.5',
        'CO': 'Flag_CO',
        'NO': 'Flag_NO'
    }
    
    # [cite_start]Rangos Físicos Válidos por Año (basado en 'Rangos...SIMA.pdf') [cite: 6]
    rangos_por_anio = {
        2020: {'PM2.5': (0, 205.94), 'CO': (0, 20), 'NO': (0, 500)},
        2021: {'PM2.5': (0, 325), 'CO': (0, 10), 'NO': (0, 350)},
        2022: {'PM2.5': (0, 450), 'CO': (0, 8), 'NO': (0, 400)},
        2023: {'PM2.5': (0, 800), 'CO': (0, 14), 'NO': (0, 500)},
        2024: {'PM2.5': (0, 999), 'CO': (0, 18), 'NO': (0, 400)},
        2025: {'PM2.5': (0, 350), 'CO': (0, 10), 'NO': (0, 350)}
    }

    df_limpio = df.copy()
    
    # Extraer el año del índice para el filtro de rangos
    df_limpio['year'] = df_limpio.index.year

    # --- 2. Aplicar Filtro de Banderas (Paso 1) ---
    print("  Aplicando filtro de banderas inválidas...")
    for var, col_flag in mapeo_variables_flags.items():
        col_limpia = f'{var}_limpio'
        
        # Lógica: Mantener el valor si la bandera NO es inválida.
        # Si ES inválida, reemplazar con Nulo (np.nan).
        df_limpio[col_limpia] = np.where(
            df_limpio[col_flag].isin(banderas_invalidas), # Condición
            np.nan,                                      # Valor si es True (inválida)
            df_limpio[var]                               # Valor si es False (válida)
        )

    # --- 3. Aplicar Filtro de Rangos Físicos (Paso 2) ---
    print("  Aplicando filtro de rangos físicos por año...")
    
    # Iteramos por cada año y aplicamos sus rangos específicos
    for anio, rangos in rangos_por_anio.items():
        # Máscara booleana para todas las filas de este año
        filtro_anio = (df_limpio['year'] == anio)
        
        for var, (min_val, max_val) in rangos.items():
            col_limpia = f'{var}_limpio'
            
            # Máscara booleana para valores fuera de rango
            filtro_rango = (
                (df_limpio[col_limpia] < min_val) | 
                (df_limpio[col_limpia] > max_val)
            )
            
            # Aplicar: Poner NaN donde el año coincide Y el rango no coincide
            # Usamos .loc para modificar el DataFrame en el lugar correcto
            df_limpio.loc[filtro_anio & filtro_rango, col_limpia] = np.nan

    # Limpiar columnas auxiliares
    df_limpio = df_limpio.drop(columns=['year'])
    
    print("Limpieza de calidad completada.")
    return df_limpio

In [14]:
# --- 1. Configuración del Análisis ---

# Define tus estaciones de interés
ESTACIONES = ['CE', 'NTE', 'SE']

# Define TODAS las columnas que necesitas (variables Y sus flags)
# !!IMPORTANTE!!: Debes verificar los nombres exactos de las columnas en tus CSV
COLUMNAS = [
    'fecha', 'estacion', 
    'PM2.5', 'Flag_PM2.5',  # Asumo este nombre para el flag
    'CO', 'Flag_CO',        # Asumo este nombre para el flag
    'NO', 'Flag_NO'         # Asumo este nombre para el flag
    # Agrega aquí las covariables meteorológicas si las necesitas (ej. 'WSR', 'Flag_WSR')
]

# Ruta a la carpeta donde están tus archivos CSV
RUTA_DATOS = "data/raw"


# --- 2. Ejecución del Proceso ---

# Paso 1: Cargar y consolidar
df_consolidado = cargar_y_consolidar_datos(
    ruta_carpeta=RUTA_DATOS,
    estaciones_interes=ESTACIONES,
    columnas_interes=COLUMNAS
)

# Verificar si la carga fue exitosa
if not df_consolidado.empty:
    
    # Paso 2: Aplicar la limpieza de calidad
    df_final_limpio = aplicar_filtros_de_calidad(df_consolidado)

    # --- 3. Verificación ---
    print("\n--- Verificación de Limpieza ---")
    
    # Compara la variable original vs. la limpia para PM2.5
    print("Datos PM2.5 (Originales):")
    print(df_final_limpio['PM2.5'].describe())
    
    print("\nDatos PM2.5 (Limpios):")
    # La media debería cambiar y la cuenta (count) debería ser menor
    print(df_final_limpio['PM2.5_limpio'].describe())

    # El DataFrame 'df_final_limpio' está ahora listo para la Fase 2 (Agregación)

    

Iniciando carga de datos de: data/raw
Error: No se encontraron archivos .csv en la ruta especificada.
